## Setup

In [21]:
import numpy as np
import pandas as pd
from ax_utils import sobol_method,ehvi_method,parego_method,get_summary_df
from ax_utils import get_init_hv_list,plot_mohpo_methods
from ax_experiments_functions import ax_rdc_setup, get_glch_hv_list_rdc


## My Dataset

In [38]:
csv_path = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/"+\
"bpp-mse-psnr-loss-flops-params_bmshj2018-factorized_10000-epochs_D-3-4_L-2e-2-1e-2-5e-3_N-32-64-96-128-160-192-224_M-32-64-96-128-160-192-224-256-288-320.csv"
search_space,optimization_config,MAX_HV = ax_rdc_setup(csv_path)

## Define experiment creation utilities

These construct our experiment, then initialize with Sobol points before we fit a Gaussian Process model to those initial points.

In [3]:
# Reasonable defaults for number of quasi-random initialization points and for subsequent model-generated trials.
SEED = 1234
N_INIT = 6
N_BATCH = 105

# Sobol
We use quasirandom points as a fast baseline for evaluating the quality of our multi-objective optimization algorithms.

In [37]:
sobol_hv_list = sobol_method(search_space,optimization_config,SEED,N_INIT,N_BATCH)

## qNEHVI
Noisy Expected Hypervolume Improvement. This is our current recommended algorithm for multi-objective optimization.

In [36]:
ehvi_hv_list = ehvi_method(search_space,optimization_config,SEED,N_INIT,N_BATCH)

## qNParEGO
This is a good alternative algorithm for multi-objective optimization when qNEHVI runs too slowly.

In [34]:
parego_hv_list = parego_method(search_space,optimization_config,SEED,N_INIT,N_BATCH)

## GLCH

In [7]:
glch_csv_name = \
"/home/lucas/Documents/perceptronac/scripts/tradeoffs/glch_results/threed_hull_bpp_loss_vs_mse_loss_vs_params_start_left.csv"

In [33]:
glch_hv_list = get_glch_hv_list_rdc(search_space,optimization_config,glch_csv_name)

# Hypervolume statistics
The hypervolume of the space dominated by points that dominate the reference point.

In [32]:
init_hv_list = get_init_hv_list(search_space,optimization_config,SEED,N_INIT)
iters = np.arange(1, N_INIT + N_BATCH + 1)
methods_df = get_summary_df(iters,init_hv_list,sobol_hv_list,ehvi_hv_list,parego_hv_list,glch_hv_list,MAX_HV)
methods_df.to_csv(f"mohpo_methods_seed{SEED}.csv")
methods_df = pd.read_csv("mohpo_methods_seed1234_b.csv")
plot_mohpo_methods(methods_df)